### load

In [2]:
## input: 1. summarizedExperiement(SE) obj for chromVAR 2. Jaspar matrix 
## output: 1. motif x cell (z score) 2. plot: ranked           
source("./libs.R")

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 772 rows [3, 50, 83, 84, 138, 181, 196, 213, 253, 265, 274, 301, 303, 353, 371, 388, 412, 434, 491, 495, ...].”

In [3]:
input.umap.res%>%head(1)

barcodes,UMAP1,UMAP2,cluster,cell_type_overall,subtype,unique_usable_reads,log_usable_counts,frac_duplicated_reads,frac_mito_reads,frac_promoters_used,frac_reads_in_peaks,frac_reads_in_promoters
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Islet1fresh_AGACACCTAAGAGGCAAAGGAGTAGGCTC,-5.102393,1.38682,alpha_2,alpha,2,6254,7.865955,0.7902986,0.03428042,0.04337516,0.5978574,0.2646306


In [4]:
table(input.umap.res %>% filter(barcodes %in% colnames(input.chromVar.jaspar.z)[-1]) %>% 
    pull(cell_type_overall))

table(input.umap.res %>% pull(cell_type_overall))


     acinar       alpha        beta       delta      ductal endothelial 
         46        6217        7598         710          80         118 
      gamma      immune    stellate 
        260         140         128 


     acinar       alpha        beta       delta      ductal endothelial 
         46        6218        7598         710          80         118 
      gamma      immune    stellate 
        260         140         128 

###  T test (sub vs. sub)

In [5]:
input.chromVar.jaspar.z <- data.table(assays(input.chromVar.res.list$dev)$z, keep.rownames = T)

# aggregate data --------------------------------------------------------------
# melt
input.chromVar.jaspar.z.agg <- melt(input.chromVar.jaspar.z, id = "rn", variable.name = "barcodes", 
    value.name = "zval")

# add celltype
input.chromVar.jaspar.z.agg <- merge(input.chromVar.jaspar.z.agg, input.umap.res)

table(input.chromVar.jaspar.z.agg%>%pull(cell_type_overall))


     acinar       alpha        beta       delta      ductal endothelial 
      26634     3600222     4399242      411090       46320       68322 
      gamma      immune    stellate 
     150540       81060       74112 

In [6]:
input.chromVar.jaspar.z.agg %>% dim
input.chromVar.jaspar.z.agg %>% head(1)
input.chromVar.jaspar.z.agg %>% filter(cell_type_overall %in% c("alpha", "beta", 
    "delta")) %>% dim
input.chromVar.jaspar.z.agg %>% filter(cell_type_overall %in% c("alpha", "beta", 
    "delta")) %>% pull(cluster) %>% table
input.chromVar.jaspar.z.agg %>% filter(cell_type_overall %in% c("alpha", "beta", 
    "delta")) %>% select(barcodes, cluster) %>% unique %>% pull(cluster) %>% table
input.chromVar.jaspar.z.agg <- input.chromVar.jaspar.z.agg %>% filter(cell_type_overall %in% c("alpha", "beta", 
    "delta")) 

[1] 8857542      15

barcodes,rn,zval,UMAP1,UMAP2,cluster,cell_type_overall,subtype,unique_usable_reads,log_usable_counts,frac_duplicated_reads,frac_mito_reads,frac_promoters_used,frac_reads_in_peaks,frac_reads_in_promoters
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Islet1fresh_AGACACCTAAGAGGCAAAGGAGTAGGCTC,MA0004.1_Arnt,-1.566904,-5.102393,1.38682,alpha_2,alpha,2,6254,7.865955,0.7902986,0.03428042,0.04337516,0.5978574,0.2646306


[1] 8410554      15

.
alpha_1 alpha_2  beta_1  beta_2 delta_1 delta_2 
3005589  594633 2434116 1965126  276762  134328 

.
alpha_1 alpha_2  beta_1  beta_2 delta_1 delta_2 
   5191    1027    4204    3394     478     232 

In [7]:
input.chromVar.jaspar.z.agg <- input.chromVar.jaspar.z.agg %>% select(rn, zval, cluster, 
    cell_type_overall)
range(input.chromVar.jaspar.z.agg$zval)
input.chromVar.jaspar.z.agg$zval[is.na(input.chromVar.jaspar.z.agg$zval)] <- 0
range(input.chromVar.jaspar.z.agg$zval)
sum(!complete.cases(input.chromVar.jaspar.z.agg))
dim(input.chromVar.jaspar.z.agg)
head(input.chromVar.jaspar.z.agg, 1)
table(input.chromVar.jaspar.z.agg %>% pull(cluster))

[1] NA NA

[1] -20.16193  31.35658

[1] 0

[1] 8410554       4

rn,zval,cluster,cell_type_overall
<chr>,<dbl>,<chr>,<chr>
MA0004.1_Arnt,-1.566904,alpha_2,alpha



alpha_1 alpha_2  beta_1  beta_2 delta_1 delta_2 
3005589  594633 2434116 1965126  276762  134328 

In [8]:
(test.all <- list(alpha = c("alpha_1", "alpha_2"), beta = c("beta_1", "beta_2"), delta = c("delta_1", 
    "delta_2")))
test.motifs <- input.chromVar.jaspar.z.agg %>% pull(rn) %>% unique()
length(test.motifs)

$alpha
[1] "alpha_1" "alpha_2"

$beta
[1] "beta_1" "beta_2"

$delta
[1] "delta_1" "delta_2"

[1] 579

In [9]:
ntest=names(test.all)[1]
celltype.test <- test.all[[ntest]]
paste(celltype.test,collapse = '.vs.')

[1] "alpha_1.vs.alpha_2"

In [10]:
require(parallel)

ttest.res.ct <- do.call(rbind, lapply(names(test.all), function(ntest) {
    celltype.test <- test.all[[ntest]]
    ttest.res <- do.call(rbind, mclapply(test.motifs, function(motif) {
        pd <- input.chromVar.jaspar.z.agg %>% filter(rn == motif) %>% mutate(cluster %in% 
            celltype.test)
        test.res <- t.test(pd %>% filter(cluster == celltype.test[1]) %>% select(zval), 
            pd %>% filter(cluster == celltype.test[2]) %>% select(zval))
        (data.frame(motif = motif, mean_x = test.res$estimate[1], mean_y = test.res$estimate[2], 
            pval = test.res$p.value/2))
    }, mc.cores = 10)) %>% mutate(test = paste(celltype.test, collapse = ".vs."))
})) %>% separate(test, into = c("x", "y"), sep = ".vs.", remove = F) %>% mutate(enrichedIn = ifelse(mean_x > 
    mean_y, x, y)) %>% group_by(test) %>% mutate(FDR = p.adjust(pval, "BH"), padj = p.adjust(pval, 
    "bonferroni"))
head(ttest.res.ct, 1)

motif,mean_x,mean_y,pval,test,x,y,enrichedIn,FDR,padj
<fct>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
MA0004.1_Arnt,0.0241363,0.8311691,3.826402e-73,alpha_1.vs.alpha_2,alpha_1,alpha_2,alpha_2,2.284007e-72,2.215487e-70


In [18]:
(celltype.test.all <- ttest.res.ct %>% pull(test) %>% unique)
(ttest.res.ct %>% separate(motif, into = c("jaspar.id", "motif"), sep = "_") %>% 
    head(1) %>% left_join(fread("../figures/Fig1/subfigs/fig1E_one_vs_other.csv") %>% 
    select(jaspar.id, family.id, family.name, class.id, class.name, superclass.id, 
        superclass.name) %>% unique))[c(2, 1, 7, 8, 6, 3, 4, 9, 5, 10:17)]
ttest.res.ct <- (ttest.res.ct %>% separate(motif, into = c("jaspar.id", "motif"), 
    sep = "_") %>% left_join(fread("../figures/Fig1/subfigs/fig1E_one_vs_other.csv") %>% 
    select(jaspar.id, family.id, family.name, class.id, class.name, superclass.id, 
        superclass.name) %>% unique))[c(2, 1, 7, 8, 6, 3, 4, 9, 5, 10:17)]

[1] "alpha_1.vs.alpha_2" "beta_1.vs.beta_2"   "delta_1.vs.delta_2"

Joining, by = "jaspar.id"


motif,jaspar.id,x,y,test,mean_x,mean_y,enrichedIn,pval,FDR,padj,family.id,family.name,class.id,class.name,superclass.id,superclass.name
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>
Arnt,MA0004.1,alpha_1,alpha_2,alpha_1.vs.alpha_2,0.0241363,0.8311691,alpha_2,3.826402e-73,2.284007e-72,2.215487e-70,1.2.5,PAS,1.2,Basic helix-loop-helix factors (bHLH),1,Basic domains


Joining, by = "jaspar.id"


In [19]:
(all.test <- c("alpha_1.vs.alpha_2.1", "alpha_1.vs.alpha_2.2", "beta_1.vs.beta_2.1", 
    "beta_1.vs.beta_2.2", "delta_1.vs.delta_2.1", "delta_1.vs.delta_2.2"))
names(all.test) <- c("alpha_1", "alpha_2", "beta_1", "beta_2", "delta_1", "delta_2")

ntest = "alpha_1"
sub(".[12]$", "", all.test[[ntest]])
ttest.res.ct %>% filter(test == sub(".[12]$", "", all.test[[ntest]]))%>%head(1)
ttest.res.ct %>% filter(test == sub(".[12]$", "", all.test[[ntest]]))%>%dim

[1] "alpha_1.vs.alpha_2.1" "alpha_1.vs.alpha_2.2" "beta_1.vs.beta_2.1"  
[4] "beta_1.vs.beta_2.2"   "delta_1.vs.delta_2.1" "delta_1.vs.delta_2.2"

[1] "alpha_1.vs.alpha_2"

motif,jaspar.id,x,y,test,mean_x,mean_y,enrichedIn,pval,FDR,padj,family.id,family.name,class.id,class.name,superclass.id,superclass.name
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>
Arnt,MA0004.1,alpha_1,alpha_2,alpha_1.vs.alpha_2,0.0241363,0.8311691,alpha_2,3.826402e-73,2.284007e-72,2.215487e-70,1.2.5,PAS,1.2,Basic helix-loop-helix factors (bHLH),1,Basic domains


[1] 614  17

In [31]:
ttest.res.ct <- ttest.res.ct %>% arrange(enrichedIn, FDR, desc(abs(mean_x - mean_y)))

(all.test <- c("alpha_1.vs.alpha_2.1", "alpha_1.vs.alpha_2.2", "beta_1.vs.beta_2.1", 
    "beta_1.vs.beta_2.2", "delta_1.vs.delta_2.1", "delta_1.vs.delta_2.2"))
names(all.test) <- c("alpha_1", "alpha_2", "beta_1", "beta_2", "delta_1", "delta_2")

ttest.res.list <- lapply(names(all.test), function(ntest) ttest.res.ct %>% filter(test == 
    sub(".[12]$", "", all.test[[ntest]]) & enrichedIn == ntest))
names(ttest.res.list) <- as.character(all.test)
require(writexl)
write_xlsx(ttest.res.list, "../figures/Fig1/subfigs/fig2D_sub_vs_sub.xlsx")
system("open ../figures/Fig1/subfigs/fig2D_sub_vs_sub.xlsx")

fwrite(ttest.res.ct, "../figures/Fig1/subfigs/fig2D_sub_vs_sub.csv")
system("open ../figures/Fig1/subfigs/fig2D_sub_vs_sub.csv")

[1] "alpha_1.vs.alpha_2.1" "alpha_1.vs.alpha_2.2" "beta_1.vs.beta_2.1"  
[4] "beta_1.vs.beta_2.2"   "delta_1.vs.delta_2.1" "delta_1.vs.delta_2.2"